# Project 3: Predicting Winners in Soccer Matches

Predicting the outcome of a sporting event is a challenging task, so much so that an entire industry exists to relieve  those who believe they can do so of their money. Despite the challenging nature of the task, there are a few people who manage to make their living (and a very good living at that!) making predictions and betting on the outcomes. Your job is to try your hand at this with a basic model.

## The Data

For this project, you are going to use a large *relational database* of publicly available information on European soccer matches that is available [here](https://www.kaggle.com/hugomathien/soccer/kernels). This database is a SQLite database. A relational database is a data structure which contains several related tables, where each table is similar to a dataframe. There are a couple of advantages to having the data in a database: for one thing, you can 'peek' at the data without loading it all into memory. Also, databases tend to be strongly typed, so that you always know the type of data that is loaded in each column (though actually SQLite is much looser in this regard than other database software). 

## The Objective(s)

Below, you will tag every match with a 1 for home team win or draw and a 0 for away team win. Your goal is to build a model to predict whether the game is a home team win or draw or an away team win. You must:

1. Explore the dataset. What appears to correlate with home team wins/losses? What is missing?
2. Separate the data into a set for training (building the model) and testing. The test set size should not be more than 20% of the full dataset.
3. Build a model to predict who wins. To determine the quality of the model, assess the model by calculating the accuracy. You can write a function to do this or use scikit-learn's `accuracy_score`, which is in the `metrics` submodule.
4. Build a couple of different models (creating a different model means using a different set of predictor variables). Which performs the best on the held-out testing dataset? Bonus points for creativity and thoroughness! You must create and test a minimum of three different models for full credit.

## Teams

1. Evan Royer, Briana Garcia (Durham)
2. Evan O'Reilly, Jacob Mechaber (Durham)
3. Patrick Donovan, Kelvin Nguyen, Robert Patenaude
4. Aaron Broady, Andrew Meyer, Wesley Couturier, Ryan Kiessling
5. Angus Jameson, Danielle LeBoeuf, Craig Vignault 
6. Brent Groden, Stephen McCarter, Aurora VanDeWater

Consider using Git/Github to share and manage your work!

## Due Date

Due by Tuesday, May 2, 3:39 PM. Submit as an HTML file on myCourses.

## Final Thoughts

This is an extremely challenging prediction task. To get a sense of how hard it is, start by making a couple absolutely dumb baseline models: predicting always home team wins, for instance, or predicting wins entirely at random. Measure the accuracy of these models. As you develop more sophisticated models, refer back to these dumb ones. Did your model do better?

This database has been explored quite a bit on [kaggle.com](https://www.kaggle.com). Feel free to use what you see there.

In [1]:
'''
DO NOT CHANGE ANYTHING IN THIS CELL
YOU MUST RUN THIS CELL TO CONTINUE
'''

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

%matplotlib inline
plt.style.use('bmh')

## Getting the data out of the database

To get data from a database, you must first connect to it from Python. Once you've connected, you then need to create a cursor. A cursor is a temporary memory space where operations can be performed.

In [2]:
'''
CHANGE YOUR PATH IN THE LINE BELOW
'''

path_to_my_database = 'C:\\Users\Andrew\\Desktop\\School\\Introudction to Data Sciecne & Analytics\\database.sqlite'
conn = sqlite3.connect(path_to_my_database)  # connect to database
cur = conn.cursor()  # create cursor

## Getting A View of the Database

One way to get an overview of the entire database is to look at the code that created it. This is SQL (Structured Query Language), which you do not know yet, but which is fairly intuitive. We'll execute a command to view the contents of the sqlite_master table, which contains the code that created the underlying tables. Syntax is as follows: first execute, then fetch the results (if you don't fetch them, they exist, but they remain in the cursor's temporary workspace).

In [3]:
'''
DO NOT CHANGE ANYTHING IN THIS CELL
'''

cur.execute("select * from sqlite_master where type == 'table';").fetchall()

[('table',
  'sqlite_sequence',
  'sqlite_sequence',
  4,
  'CREATE TABLE sqlite_sequence(name,seq)'),
 ('table',
  'Player_Attributes',
  'Player_Attributes',
  11,
  'CREATE TABLE "Player_Attributes" (\n\t`id`\tINTEGER PRIMARY KEY AUTOINCREMENT,\n\t`player_fifa_api_id`\tINTEGER,\n\t`player_api_id`\tINTEGER,\n\t`date`\tTEXT,\n\t`overall_rating`\tINTEGER,\n\t`potential`\tINTEGER,\n\t`preferred_foot`\tTEXT,\n\t`attacking_work_rate`\tTEXT,\n\t`defensive_work_rate`\tTEXT,\n\t`crossing`\tINTEGER,\n\t`finishing`\tINTEGER,\n\t`heading_accuracy`\tINTEGER,\n\t`short_passing`\tINTEGER,\n\t`volleys`\tINTEGER,\n\t`dribbling`\tINTEGER,\n\t`curve`\tINTEGER,\n\t`free_kick_accuracy`\tINTEGER,\n\t`long_passing`\tINTEGER,\n\t`ball_control`\tINTEGER,\n\t`acceleration`\tINTEGER,\n\t`sprint_speed`\tINTEGER,\n\t`agility`\tINTEGER,\n\t`reactions`\tINTEGER,\n\t`balance`\tINTEGER,\n\t`shot_power`\tINTEGER,\n\t`jumping`\tINTEGER,\n\t`stamina`\tINTEGER,\n\t`strength`\tINTEGER,\n\t`long_shots`\tINTEGER,\n\t`aggr

## Getting Data From SQLite into Python

As you can see, there are tables like Player_Attributes, Player, Matches, Team_Attributes, and so on. It's easy to bring one of these tables into Pandas with a simple SQL query. In SQL, * is a stand-in for 'everything'. To read a SQL query, keep in mind that you need a connection open, and you need to specify the connection when you write the query.

In [4]:
player_attributes = pd.read_sql_query('select * from Player_Attributes', conn)  # don't forget to specify the connection
print(player_attributes.shape)
player_attributes.head()

(183978, 42)


,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [5]:
'''
DO NOT CHANGE ANYTHING IN THIS CELL
'''

players = pd.read_sql_query('select * from Player', conn)  # don't forget to specify the connection
print(players.shape)
players.head()

(11060, 7)


,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


In [6]:
matches = pd.read_sql_query('select * from Match', conn)  # specify the connection
print(matches.shape)
matches.head()

(25979, 115)


,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [7]:
league = pd.read_sql_query('select * from League', conn)  # don't forget to specify the connection
print(league.shape)
league.head()

(11, 3)


,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A


In [8]:
country = pd.read_sql_query('select * from Country', conn)  # don't forget to specify the connection
print(country.shape)
country.head()

(11, 2)


,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy


In [9]:
teams = pd.read_sql_query('select * from Team', conn)  # don't forget to specify the connection
print(teams.shape)
teams.head()

(299, 5)


,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


In [10]:
team_attributes = pd.read_sql_query('select * from Team_Attributes', conn)  # don't forget to specify the connection
print(team_attributes.shape)
team_attributes.head()

(1458, 25)


,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19 00:00:00,52,Balanced,48.0,Normal,56,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,3,434,9930,2015-09-10 00:00:00,47,Balanced,41.0,Normal,54,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,4,77,8485,2010-02-22 00:00:00,70,Fast,NaN,Little,70,Long,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,5,77,8485,2011-02-22 00:00:00,47,Balanced,NaN,Little,52,Mixed,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover


## Complex Queries

You can bring in selected subsets of the database using more complex SQL queries. If you know SQL already, this may be useful. We won't have time to go into complex SQL queries, but it's worth your while to read a little bit about the language; after all, SQL is one of the top languages for data science in its own right. See [this non-python specific SQLite tutorial](https://www.tutorialspoint.com/sqlite/) for an introduction. Fortunately, this dataset is small enough that you should be able to bring most/all of it into memory and thereby work almost entirely in Python.

## Labeling The Matches

The dataset currently doesn't contain a column stating who won and who lost. You'll need to add that. To do so, you can do something like below. You may group draws with home team wins. 

In [11]:
'''
DO NOT CHANGE ANYTHING IN THIS CELL
'''

matches['home_team_win_or_draw'] = 0
matches.loc[matches['home_team_goal'] >= matches['away_team_goal'], 'home_team_win_or_draw'] = 1

# Good Luck!

# Part 1
Explore the dataset. What appears to correlate with home team wins/losses? What is missing?

In [12]:
# your code here
target = matches[['home_team_win_or_draw']]
target.head()

,home_team_win_or_draw
0,1
1,1
2,0
3,1
4,0


In [13]:
result = team_attributes.sort('team_api_id')
print(result.shape)

(1458, 25)


C:\Users\Andrew\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [14]:
result.head()

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
1114,1115,874,1601,2012-02-22 00:00:00,53,Balanced,NaN,Little,55,Mixed,...,50,Normal,Organised,43,Medium,44,Press,49,Normal,Cover
1115,1116,874,1601,2013-09-20 00:00:00,53,Balanced,NaN,Little,55,Mixed,...,50,Normal,Organised,43,Medium,44,Press,49,Normal,Cover
1112,1113,874,1601,2010-02-22 00:00:00,30,Slow,NaN,Little,40,Mixed,...,70,Lots,Organised,65,Medium,60,Press,50,Normal,Cover
1113,1114,874,1601,2011-02-22 00:00:00,48,Balanced,NaN,Little,51,Mixed,...,51,Normal,Organised,46,Medium,48,Press,50,Normal,Cover
1117,1118,874,1601,2015-09-10 00:00:00,47,Balanced,48.0,Normal,38,Mixed,...,50,Normal,Organised,43,Medium,44,Press,49,Normal,Cover


In [15]:
new_result = result._get_numeric_data()
new_result.head()

,id,team_fifa_api_id,team_api_id,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth
1114,1115,874,1601,53,NaN,55,44,65,50,43,44,49
1115,1116,874,1601,53,NaN,55,44,65,50,43,44,49
1112,1113,874,1601,30,NaN,40,50,35,70,65,60,50
1113,1114,874,1601,48,NaN,51,68,67,51,46,48,50
1117,1118,874,1601,47,48.0,38,61,65,50,43,44,49


In [16]:
result_1 = matches.sort('home_team_api_id')
result_1.shape

C:\Users\Andrew\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


(25979, 116)

In [17]:
result_1.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,home_team_win_or_draw
16656,16657,15722,15722,2011/2012,6,2011-09-09 00:00:00,1030879,1601,8021,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
16526,16527,15722,15722,2011/2012,19,2012-02-24 00:00:00,1031175,1601,2182,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
17248,17249,15722,15722,2014/2015,19,2014-12-13 00:00:00,1802720,1601,10265,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
17087,17088,15722,15722,2013/2014,28,2014-03-28 00:00:00,1467915,1601,8033,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
16046,16047,15722,15722,2009/2010,19,2010-03-06 00:00:00,674613,1601,8021,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [18]:
win_draw_team = matches['home_team_win_or_draw'].groupby(matches['home_team_api_id'])
ratio_win_home_team = win_draw_team.apply(lambda x: x.sum() / len(x))
ratio_win_home_team.head()

home_team_api_id
1601    0.683333
1773    0.644444
1957    0.750000
2033    0.613333
2182    0.841667
Name: home_team_win_or_draw, dtype: float64

In [19]:
ratio_win_home_team.shape

(299,)

# Part 2
Separate the data into a set for training (building the model) and testing. The test set size should not be more than 20% of the full dataset.

In [20]:
# your code here
import statsmodels.api as sm
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score as acc
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error as mse
from IPython.core.pylabtools import figsize

%matplotlib inline
plt.style.use('bmh')


import warnings

warnings.filterwarnings('ignore')

In [21]:
features = matches[['home_team_api_id', 'away_team_api_id']]
target = matches['home_team_win_or_draw']

Xtr, Xte, ytr, yte = train_test_split(features, target, test_size = 0.2, random_state = 0)

lm_1 = LogisticRegression().fit(Xtr, ytr)
preds_1 = lm_1.predict(Xte)
print(mse(yte, preds_1))
print(acc(yte, preds_1))

0.291377983064
0.708622016936


In [ ]:
figsize(12,6)
plt.scatter(features, target)
plt.scatter(Xte, preds_1, color = 'red')

# Part 3
Build a model to predict who wins. To determine the quality of the model, assess the model by calculating the accuracy. You can write a function to do this or use scikit-learn's accuracy_score, which is in the metrics submodule.

In [22]:
# your code here

# Part 4
Build a couple of different models (creating a different model means using a different set of predictor variables). Which performs the best on the held-out testing dataset? Bonus points for creativity and thoroughness! You must create and test a minimum of three different models for full credit.

In [23]:
# your code here